# Ice Cap Sizes

This notebook reads the exploded data files and calculates the sizes of the ice caps and ice catchments in them and then adds those areas to the dataframe. Finally, the 10 largest ice caps in each region are saved to a shapefile.

These are the regions where ice caps are being evaluated:

* Region 3 - Arctic Canada, North
* Region 4 - Arctic Canada, South
* Region 5 - Greenland
* Region 6 - Iceland
* Region 7 - Svalbard and Jan Mayen (Note these are analyzed separately since they are far apart)
* Region 8 - Scandinavia
* Region 9 - Russian Arctic
* Region 10 - Asia, North
* Region 17 - Southern Andes

In [1]:
import os
import os.path as op
import sys
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

# set working dir
HOME = op.join(op.expanduser("~"))
os.chdir(os.path.join(HOME, "git/wgms-glacier-project"))

# Set up path to load scripts
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import scripts.wgms_scripts as ws

In [2]:
# Set up data dictionary with CRS codes for each region
crs_codes = {
            '3' : 'epsg:3347', # another possibility - 'esri:102001'
            '4' : 'epsg:3347',
            '5' : 'epsg:32627',
            '6' : 'epsg:3057',
            '7' : 'epsg:3049', # region 7 svalbard only since svalbard and jan mayan are far apart
            '7_jan_mayan' : 'epsg:3058', # region 7 jan mayan only since svalbard and jan mayan are far apart
            '8' : 'epsg:3049',
            '9' : 'epsg:5940', # this one is polar sterographic - should I use it????
            '10' : 'esri:102025', # another possibility - esri:102026
            '17' : 'esri:102033' # another possibility - esri:102032
            }

In [5]:
for region in crs_codes:
    # Open exploded region file
    print("Region: ", region)
    region_fn = "data/glims/processed/ice-caps/exploded/exploded_" + str(region) + ".shp"
    glims_region_df = gpd.read_file(region_fn)
    
    # Determine the area of all the polygons
    region_polygon_areas = glims_region_df['geometry'].to_crs({'init': crs_codes[str(region)]}).area/10**6
    
    # Add the areas to the dataframe
    glims_region_df = glims_region_df.assign(area=region_polygon_areas)
    
    # Determine the 10 largest ice caps
    ten_largest_df = glims_region_df[['id', 'area', 'geometry']].nlargest(10, 'area')
    
    # Print 10 largest and their size in km^2
    print(ten_largest_df)
    print("")
    
    # Save ten largest dataframe for each region to shapefile if it doesn't already exist
    output_fp = "data/glims/processed/ice-caps/largest/largest-ice-caps-region_" + str(region) + ".shp"
    if os.path.exists(output_fp) == False:
        ten_largest_df.to_file(driver='ESRI Shapefile', filename=output_fp)
    else:
        print("Region " + str(region) + " file has already been processed.")
        print("")
        print("")

Region:  3
        id          area                                           geometry
13      13  21159.547290  POLYGON ((-80.798525 81.250319, -80.793099 81....
9        9  19311.322286  POLYGON ((-80.374956 77.30712200000001, -80.37...
15      15  18917.749968  POLYGON ((-74.264652 79.611407, -74.2605049999...
1        1  13655.168957  POLYGON ((-81.72486000000001 74.56395999999999...
8        8   8142.302043  POLYGON ((-88.52648499999999 78.96548900000001...
2        2   5296.297354  POLYGON ((-80.81959999999999 76.21677, -80.824...
937    937   2824.569443  POLYGON ((-91.178678 78.33252, -91.18216099999...
444    444   2628.430841  POLYGON ((-85.331221 76.470753, -85.33386 76.4...
7        7   1871.517710  POLYGON ((-79.291584 79.236374, -79.282926 79....
1690  1690   1769.351275  POLYGON ((-84.797616 80.837233, -84.800308 80....

Region 3 file has already been processed.


Region:  4
        id         area                                           geometry
1771  1771  6133.37213

# Test area

In [ ]:
'''region=17
region_fn = "data/glims/processed/ice-caps/exploded/exploded_" + str(region) + ".shp"
glims_region_df = gpd.read_file(region_fn)
    
# Determine the area of all the polygons
region_polygon_areas = glims_region_df['geometry'].to_crs({'init': crs_codes[str(region)]}).area/10**6
    
# Add the areas to the dataframe
glims_region_df = glims_region_df.assign(area=region_polygon_areas)
    
# Determine the 10 largest ice caps
ten_largest_df = glims_region_df[['id', 'area', 'geometry']].nlargest(10, 'area')
    
# Print 10 largest and their size in km^2
print(ten_largest_df)

# Save regional dataframe to shapefile
fp = "data/glims/processed/ice-caps/largest/largest-ice-caps-region_" + str(region) + ".shp"
ten_largest_df.to_file(driver='ESRI Shapefile', filename=fp)'''